In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.

In [14]:
#pip install clean-text

In [2]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [3]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [4]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (228 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 121747 

In [5]:
!pip install poppler-utils

In [6]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [7]:
!rm -rf "db"

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
pdf_file_path = '/content/drive/MyDrive/povs/'
loader = PyPDFDirectoryLoader(pdf_file_path)
docs = loader.load()
len(docs)

22

In [10]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

60

In [12]:
texts

[Document(page_content='An integrated approach to \ncombat cyber risk\nSecuring industrial operations \nin\xa0oil and gas', metadata={'source': '/content/drive/MyDrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 0}),
 Document(page_content='ContentsForeword  1\nIntroduction  2\nUnderstanding the risks  4\nConduct a\xa0maturity assessment  6\nBuild a\xa0unified program  8\nImplement key controls  9\nEmbrace good governance  10\nConclusion  11\nContact us  12', metadata={'source': '/content/drive/MyDrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 1}),
 Document(page_content='Foreword\nThe oil and gas industry is moving into the \nnext stage of evolution, whereby robotics, \ndigitization, and the Internet of Things \n(IoT) are rapidly being integrated into the \noperational environment. The\xa0interest of \ncyber criminals in industrial operations has \nincreased over the last decade resulting in \ncyberattacks that have compromised both \nprod

In [15]:
from cleantext import clean

In [23]:
clean_texts=clean(texts)

In [24]:
len(clean_texts)

55472

In [19]:
cleaned_texts

'[document(page_content=\'an integrated approach to\ncombat cyber risk\nsecuring industrial operations\ninoil and gas\', metadata={\'source\': \'/content/drive/mydrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf\', \'page\': 0}), document(page_content=\'contentsforeword 1\nintroduction 2\nunderstanding the risks 4\nconduct amaturity assessment 6\nbuild aunified program 8\nimplement key controls 9\nembrace good governance 10\nconclusion 11\ncontact us 12\', metadata={\'source\': \'/content/drive/mydrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf\', \'page\': 1}), document(page_content=\'foreword\nthe oil and gas industry is moving into the\nnext stage of evolution, whereby robotics,\ndigitization, and the internet of things\n(iot) are rapidly being integrated into the\noperational environment. theinterest of\ncyber criminals in industrial operations has\nincreased over the last decade resulting in\ncyberattacks that have compromised both\nproduction and safety. 

In [20]:
import re

# Define a function to remove any non-alphabetic characters from the text
def clean_text(text):
  # Replace any newline characters with spaces
  text = text.replace("\n", " ")
  # Replace any non-breaking spaces with spaces
  text = text.replace("\xa0", " ")
  # Use a regular expression to find and remove any non-alphabetic characters
  text = re.sub("[^a-zA-Z ]", "", text)
  # Return the cleaned text
  return text

# Call the function and print the result
cleaned_texts = clean_text(cleaned_texts)
print(cleaned_texts)


documentpagecontentan integrated approach to combat cyber risk securing industrial operations inoil and gas metadatasource contentdrivemydrivepovsintergratedapproachcombatcyberriskoilgaspdf page  documentpagecontentcontentsforeword  introduction  understanding the risks  conduct amaturity assessment  build aunified program  implement key controls  embrace good governance  conclusion  contact us  metadatasource contentdrivemydrivepovsintergratedapproachcombatcyberriskoilgaspdf page  documentpagecontentforeword the oil and gas industry is moving into the next stage of evolution whereby robotics digitization and the internet of things iot are rapidly being integrated into the operational environment theinterest of cyber criminals in industrial operations has increased over the last decade resulting in cyberattacks that have compromised both production and safety theseattacks have made cyber security ahot discussion topic in boardrooms around the world and now agrowing number of organizati

In [26]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 4.33 s, sys: 72.8 ms, total: 4.4 s
Wall time: 5.46 s


## Llama 2-13B

In [27]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

In [28]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [30]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=10,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [31]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 10})

In [32]:

DEFAULT_SYSTEM_PROMPT = """
You are a consultant from Deloitte. Answer the questions with details related to the contents found in the text files.""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

In [34]:
result = qa_chain("What is deloitte's insight about From bytes to barrels?")

 I cannot determine Deloitte's insights about "From Bytes to Barrels" based on the provided text. The text does not mention "From Bytes to Barrels" or any similar phrase. Therefore, I cannot provide an answer to the question.


In [36]:


result = qa_chain("who to be contacted if there is any question about integrated approach to combat cyber risk   article?")

 Based on the text provided, there is no clear information about who to contact if there are any questions about the integrated approach to combat cyber risk. However, since the article mentions "practical steps for getting started" and "expanding cyber security programs," it suggests that the author or the organization providing the article may have resources or experts available to assist with implementing an integrated approach to cyber security in the oil and gas industry.


In [37]:
len(result["source_documents"])

2

In [38]:
print(result["source_documents"][1].page_content)


inherently different, requiring engineering 
know‑how, and not just IT expertise, in 
order to secure them appropriately.
Today, an approach that brings together 
IT and engineering is needed to address 
cyber security programmatically and 
sustainably. The following discusses 
the goals of such an approach as well 
as practical steps for getting started. 
First, let’s take a closer look at the types of 
cyber risks facing the oil and gas industry, 
how they can disrupt the value chain, and 
what the consequences could be.
 While the industry 
has escaped a major 
operational catastrophe 
thus far, this good fortune 
may not last unless 
companies expand their 
cyber security programs.Introduction
2An integrated approach to combat cyber risk  | Securing industrial operations in oil and gas


In [39]:
result=qa_chain("What are the challenges of making operational processes secure, vigilant and resilient in the oil and gas industry?")

 Based on the provided context, I can identify the following challenges of making operational processes secure, vigilant, and resilient in the oil and gas industry:

1. Aging infrastructure: Many oil and gas facilities have aging infrastructure, which can create vulnerabilities that can be exploited by attackers. Securing these systems can be challenging due to the lack of support for modern security protocols and the difficulty of integrating new technologies with legacy systems.
2. Remote locations: Oil and gas facilities are often located in remote areas, making it difficult to maintain a constant physical presence and increasing the likelihood of unauthorized access.
3. Limited resources: Smaller oil and gas companies may not have the financial or human resources to invest in comprehensive security measures, leaving them more vulnerable to attacks.
4. Complexity of industrial control systems: Industrial control systems (ICS) used in oil and gas operations are highly complex and cus

## Prompts for evaluation

1- What are the main drivers for the shift to integrated reporting across industries and sectors?

2- How does integrated reporting address the disconnect between what companies are disclosing and what stakeholders expect?

3- What are the key components of the Integrated Reporting Framework and how do they help to communicate the value creation of an organization?

4- What are some examples of factors that can affect an organization’s ability to create long-term value for its stakeholders?

6- What are the main reasons why operational systems were not designed with security in mind?

7- How does the IoT increase the vulnerability of industrial facilities to cyber attacks?

8- What are the benefits of adopting an integrated approach to cyber security that brings together IT and engineering?

9- What are some of the goals and practical steps of such an approach?

How does cyber security affect the value creation and preservation of the oil and gas industry?

In [40]:
result=qa_chain("What are the main drivers for the shift to integrated reporting across industries and sectors?")

 Based on the provided context, I cannot determine the specific drivers for the shift to integrated reporting across industries and sectors. The text mentions a disconnect between what companies are disclosing, what investors expect, and what broader stakeholder groups expect, but does not explicitly state the main drivers for the shift to integrated reporting. Additionally, the text emphasizes the importance of meeting stakeholder demands and creating a foundation for future standards, which suggests that the shift to integrated reporting may be driven by a desire to address stakeholder needs and expectations more effectively. However, without further information, it is difficult to pinpoint the exact drivers for this shift.


In [42]:
result=qa_chain("How does integrated reporting address the disconnect between what companies are disclosing and what stakeholders expect?")

 Based on the provided context, I cannot determine how integrated reporting addresses the disconnect between what companies are disclosing and what stakeholders expect. The text mentions that there is a disconnect between what companies are disclosing, what investors expect, and what broader stakeholder groups expect, but it does not provide specific information on how integrated reporting addresses this issue. Therefore, I do not know the answer to the question.


In [43]:
result=qa_chain("What are the key components of the Integrated Reporting Framework and how do they help to communicate the value creation of an organization?")

 Based on the provided context, I can answer the question. Here are the key components of the Integrated Reporting Framework and how they help to communicate the value creation of an organization:

Key Components of the Integrated Reporting Framework:

1. Principles-based guidance for internal and external reporting structure and content.
2. Includes both financial and non-financial value drivers.
3. Founded on integrated thinking, which is based on integrated decision-making and actions that combine to affect the value creation of an organization.

How the Key Components Help Communicate Value Creation:

The Integrated Reporting Framework provides a comprehensive and integrated approach to communicating an organization's value creation. By including both financial and non-financial value drivers, the framework helps to provide a more complete picture of an organization's performance and value creation. The principles-based guidance for reporting structure and content helps to ensure t

In [44]:
result=qa_chain("What are some examples of factors that can affect an organization’s ability to create long-term value for its stakeholders?")

 Based on the provided context, here are some examples of factors that can affect an organization's ability to create long-term value for its stakeholders:

1. Integrated thinking and decision-making: The ability to integrate financial and non-financial metrics and drivers into a single reporting framework can help organizations better communicate their value creation processes to stakeholders.
2. Value creation models: Developing a clear understanding of how the organization creates value for its stakeholders through a value creation model can help identify areas where improvements can be made to enhance long-term value creation.
3. Performance measurement: Effective measurement and reporting of performance against strategic objectives can help organizations understand how they are creating value and identify areas for improvement.
4. Risk management: Identifying and managing risks that could impact the organization's ability to create long-term value is crucial to achieving long-term

In [45]:
result=qa_chain("What are the main reasons why operational systems were not designed with security in mind?")

 Based on the provided text, the main reasons why operational systems were not designed with security in mind are:

1. Little need for it: In the past, operational systems were not integrated with enterprise systems or even with each other, so the risk of large-scale cascading failures due to cyber attacks was extremely isolated.
2. Fit-for-purpose, isolated operational systems: Operational systems were designed to meet specific needs and were not intended to be connected to external networks, which reduced the need for security measures.
3. Limited connectivity: The IoT has turned the basic assumptions about operational security upside down, making all sorts of industrial facilities vulnerable to cyber attacks, regardless of their location.


In [46]:
result=qa_chain("How does the IoT increase the vulnerability of industrial facilities to cyber attacks?")

 Based on the provided text, the IoT increases the vulnerability of industrial facilities to cyber attacks by connecting operational systems to the internet, allowing external risks to compromise safety and production.


In [47]:
result=qa_chain("What are the benefits of adopting an integrated approach to cyber security that brings together IT and engineering?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Based on the provided context, I can answer the question. Here's my response:

The benefits of adopting an integrated approach to cybersecurity that brings together IT and engineering include:

1. Programmatic and sustainable security: By integrating IT and engineering, organizations can ensure that their cybersecurity efforts are comprehensive, coordinated, and long-term, rather than piecemeal and reactive.
2. Appropriate risk management: Engineering expertise is essential to understand the specific risks and vulnerabilities associated with industrial operations in the oil and gas sector. An integrated approach ensures that these risks are properly assessed and addressed.
3. Prevention of operational catastrophes: Cyber attacks can cause significant disruptions to industrial operations, leading to costly downtime, environmental damage, and even loss of human life. An integrated approach helps prevent such operational catastrophes by proactively identifying and mitigating potential th